In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import noise_correlations
import numpy as np
import os
import pickle

from importlib import reload
from neuropacks import CV
from noise_correlations import (analysis,
                                discriminability,
                                null_models,
                                utils,
                                plot)

from mpl_lego import colorbar as pcb
from mpl_lego import ellipse

%matplotlib widget

In [ ]:
plt.rcParams.update({'text.usetex': True, 'font.family': 'serif'})

In [ ]:
# Load PVC11 dataset
# base_path = '/Volumes/pss/data/pvc11/data/spikes_gratings'
base_path = '/storage/data/cv_perception'
data_path = os.path.join(base_path, 'cv_all.h5')
pack = CV(data_path)

In [ ]:
# Get response matrix for PVC11 dataset, and stimuli
X = pack.get_response_matrix(stimulus='c', norm=True)
stimuli = pack.get_design_matrix(stimulus='c')
unique_stimuli = np.unique(stimuli)
stimuli = np.array([np.argwhere(unique_stimuli == stim).item()
                    for stim in stimuli])

In [ ]:
# Random state
rng = np.random.default_rng()

In [ ]:
dims = np.array([3])
n_dims = dims.size
n_dimlets = 10
n_repeats = 1000
n_stims_per_dimlet = 10
n_pairings = n_dimlets * n_stims_per_dimlet
v_s_lfis = np.zeros((n_dims, n_pairings, n_repeats))
v_s_sdkls = np.zeros_like(v_s_lfis)
v_r_lfis = np.zeros_like(v_s_lfis)
v_r_sdkls = np.zeros_like(v_s_lfis)
v_lfis = np.zeros((n_dims, n_pairings))
v_sdkls = np.zeros_like(v_lfis)
units = {idx: np.zeros((n_pairings, dim)) for idx, dim in enumerate(dims)}
stims = np.zeros((n_dims, n_pairings, 2))

In [ ]:
# Calculate null measures across dims
for idx, dim in enumerate(dims):
    print(idx)
    v_s_lfis[idx], v_s_sdkls[idx], v_r_lfis[idx], v_r_sdkls[idx], \
    v_lfis[idx], v_sdkls[idx], units[idx], stims[idx] = \
        analysis.calculate_null_measures(
            X=X,
            stimuli=stimuli,
            n_dim=dim,
            n_dimlets=n_dimlets,
            rng=rng,
            n_repeats=n_repeats,
            circular_stim=False,
            unordered=True,
            all_stim=False,
            n_stims_per_dimlet=n_stims_per_dimlet,
            verbose=True)

In [ ]:
# Calculate percentiles
p_r_lfi = 100 * np.mean(v_lfis[..., np.newaxis] > v_r_lfis, axis=2)
p_r_sdkl = 100 * np.mean(v_sdkls[..., np.newaxis] > v_r_sdkls, axis=2)

In [ ]:
# Second plot
rot_idx = 7
stim0_idx = np.argwhere(stimuli == stims[0, rot_idx][0]).ravel()
stim1_idx = np.argwhere(stimuli == stims[0, rot_idx][1]).ravel()
X0 = X[stim0_idx][:, units[0][rot_idx]]
X1 = X[stim1_idx][:, units[0][rot_idx]]

mu0, cov0 = utils.mean_cov(X0)
mu1, cov1 = utils.mean_cov(X1)
fpr = mu1 - mu0
mu_mean = (mu0 + mu1) / 2
cov_avg = (cov0 + cov1) / 2
_, vs = np.linalg.eigh(cov_avg)
v_small = vs[:, 0]
fpr_unit = fpr / np.linalg.norm(fpr)

R = utils.get_rotation_for_vectors(v_small, fpr_unit)
best_cov = R @ cov_avg @ R.T

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

ellipse.plot_cov_ellipse(cov0, alpha=0.65, mu=mu0, ax=ax, fc='blue')
ellipse.plot_cov_ellipse(cov1, alpha=0.65, mu=mu1, ax=ax, fc='orange')
#ellipse.plot_cov_ellipse(cov_avg, alpha=0.5, mu=mu_mean, ax=ax, fc='red')
#ellipse.plot_cov_ellipse(best_cov, alpha=0.5, mu=mu_mean, ax=ax, fc='green')

ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.axvline(0, linestyle='--', color='gray')
ax.axhline(0, linestyle='--', color='gray')
ax.scatter(X0[:, 0], X0[:, 1], color=u'#1f77b4', alpha=0.5)
ax.scatter(X1[:, 0], X1[:, 1], color=u'#ff7f0e', alpha=0.5)
ax.tick_params(labelsize=14)
#ax.arrow(mu0[0], mu0[1], mu1[0] - mu0[0], mu1[1] - mu0[1], head_width=0.5, shape='full')
ax.set_xlabel(r'\textbf{Neuron 1}', fontsize=20)
ax.set_ylabel(r'\textbf{Neuron 2}', fontsize=20)
# plt.savefig('nc_example_neuron.pdf', bbox_inches='tight')
plt.show()

In [ ]:
p_r_sdkl

In [ ]:
# Second plot
rot_idx = 1
stim0_idx = np.argwhere(stimuli == stims[0, rot_idx][0]).ravel()
stim1_idx = np.argwhere(stimuli == stims[0, rot_idx][1]).ravel()
X0 = X[stim0_idx][:, units[0][rot_idx]]
X1 = X[stim1_idx][:, units[0][rot_idx]]
mu0, cov0 = utils.mean_cov(X0)
mu1, cov1 = utils.mean_cov(X1)

pair = [1, 2]
X0 = X0[:, pair]
X1 = X1[:, pair]
mu0 = mu0[pair]
mu1 = mu1[pair]
cov0 = cov0[pair][:, pair]
cov1 = cov1[pair][:, pair]

fpr = mu1 - mu0
mu_mean = (mu0 + mu1) / 2
cov_avg = (cov0 + cov1) / 2
_, vs = np.linalg.eigh(cov_avg)
v_small = vs[:, 0]
fpr_unit = fpr / np.linalg.norm(fpr)

R = utils.get_rotation_for_vectors(v_small, fpr_unit)
best_cov = R @ cov_avg @ R.T

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

ellipse.plot_cov_ellipse(cov0, alpha=0.65, mu=mu0, ax=ax, fc='blue')
ellipse.plot_cov_ellipse(cov1, alpha=0.65, mu=mu1, ax=ax, fc='orange')
#ellipse.plot_cov_ellipse(cov_avg, alpha=0.5, mu=mu_mean, ax=ax, fc='red')
#ellipse.plot_cov_ellipse(best_cov, alpha=0.5, mu=mu_mean, ax=ax, fc='green')

ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.axvline(0, linestyle='--', color='gray')
ax.axhline(0, linestyle='--', color='gray')
ax.scatter(X0[:, 0], X0[:, 1], color=u'#1f77b4', alpha=0.5)
ax.scatter(X1[:, 0], X1[:, 1], color=u'#ff7f0e', alpha=0.5)
ax.tick_params(labelsize=14)
#ax.arrow(mu0[0], mu0[1], mu1[0] - mu0[0], mu1[1] - mu0[1], head_width=0.5, shape='full')
ax.set_xlabel(r'\textbf{Neuron 1}', fontsize=20)
ax.set_ylabel(r'\textbf{Neuron 2}', fontsize=20)
# plt.savefig('nc_example_neuron.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Second plot
rot_idx = 11
stim0_idx = np.argwhere(stimuli == stims[0, rot_idx][0]).ravel()
stim1_idx = np.argwhere(stimuli == stims[0, rot_idx][1]).ravel()
X0 = X[stim0_idx][:, units[0][rot_idx]]
X1 = X[stim1_idx][:, units[0][rot_idx]]
mu0, cov0 = utils.mean_cov(X0)
mu1, cov1 = utils.mean_cov(X1)


fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')
ax.scatter(X0[:, 0], X0[:, 1], X0[:, 2])
ax.scatter(X1[:, 0], X1[:, 1], X1[:, 2])

In [ ]:
avg_corr = np.zeros((n_dims - 1, n_pairings))
avg_corr_best = np.zeros((n_dims - 1, n_pairings))

for idx, dim_idx in enumerate([1, 2, 3]):
    for pairing in range(n_pairings):
        stim0_idx = np.argwhere(stimuli == stims[dim_idx, pairing][0]).ravel()
        stim1_idx = np.argwhere(stimuli == stims[dim_idx, pairing][1]).ravel()
        X0 = X[stim0_idx][:, units[dim_idx][pairing]]
        X1 = X[stim1_idx][:, units[dim_idx][pairing]]

        mu0, cov0 = utils.mean_cov(X0)
        mu1, cov1 = utils.mean_cov(X1)
        fpr = mu1 - mu0
        avg_cov = (cov0 + cov1) / 2
        _, vs = np.linalg.eigh(avg_cov)
        v_small = vs[:, 0]
        fpr_unit = fpr / np.linalg.norm(fpr)

        R = utils.get_rotation_for_vectors(v_small, fpr_unit)
        best_cov = R @ avg_cov @ R.T


        avg_corr[idx, pairing] = np.mean(
            [np.corrcoef(np.diag(avg_cov), mu0)[0, 1],
             np.corrcoef(np.diag(avg_cov), mu1)[0, 1]]
        )
        avg_corr_best[idx, pairing] = np.mean(
            [np.corrcoef(np.diag(best_cov), mu0)[0, 1],
             np.corrcoef(np.diag(best_cov), mu1)[0, 1]]
        )
        

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharey=True)

for idx, ax in enumerate(axes):
    ax.scatter(avg_corr_best[idx], p_r_lfi[idx + 1],
               color='k',
               alpha=0.25)
    ax.set_ylim([-1, 100])
    ax.set_xlim([-1, 1])
    ax.set_title(f'$D={dims[idx + 1]}$')

axes[0].set_xlabel(r'\textbf{Correlation btw mean/var}', fontsize=20)
axes[0].set_ylabel(r'\textbf{Percentile}', fontsize=20)
    
plt.tight_layout()
plt.savefig('nc_corr_dim.pdf', bbox_inches='tight')

In [ ]:
min_best = np.zeros((n_dims, n_pairings))

for dim_idx in range(n_dims):
    for pairing in range(n_pairings):
        # Get subset of design matrices
        stim0_idx = np.argwhere(stimuli == stims[dim_idx, pairing][0]).ravel()
        stim1_idx = np.argwhere(stimuli == stims[dim_idx, pairing][1]).ravel()
        X0 = X[stim0_idx][:, units[dim_idx][pairing]]
        X1 = X[stim1_idx][:, units[dim_idx][pairing]]
        # Calculate means and covariances
        mu0, cov0 = utils.mean_cov(X0)
        mu1, cov1 = utils.mean_cov(X1)
        # Derivative of tuning direction
        fpr = mu1 - mu0
        fpr_unit = fpr / np.linalg.norm(fpr)
        # Get observed and best covariance matrices
        avg_cov = (cov0 + cov1) / 2
        v_small = np.linalg.eigh(avg_cov)[1][:, 0]
        R = utils.get_rotation_for_vectors(v_small, fpr_unit)
        best_cov = R @ avg_cov @ R.T
        
        # Get principal eigenvectors of best covariance matrix
        u_big_best = np.sqrt(np.linalg.eigh(best_cov)[0][-1])
        v_big_best = np.linalg.eigh(best_cov)[1][:, -1]
        min_best[dim_idx, pairing] = np.min((mu0 - u_big_best * v_big_best,
                                             mu0 + u_big_best * v_big_best,
                                             mu1 - u_big_best * v_big_best,
                                             mu1 + u_big_best * v_big_best))


In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 4), sharey=True)

for idx, ax in enumerate(axes):
    ax.scatter(min_best[idx], p_r_lfi[idx],
               color='k',
               alpha=0.25)
    ax.set_ylim([-1, 100])
    ax.set_xlim([-20, 10])
    ax.set_title(f'$D={dims[idx]}$')

axes[0].set_xlabel(r'\textbf{Min. value in neural space}', fontsize=20)
axes[0].set_ylabel(r'\textbf{Percentile}', fontsize=20)
    
plt.tight_layout()
plt.savefig('nc_neural_space_dim.pdf', bbox_inches='tight')